# 데이터 분리

In [1]:
# TrainValid 파라미터에 따라 경로 자동 설정
TrainValid = "Validation"  # "Training" 또는 "Validation" 중 선택

if TrainValid == "Training":
    base_path = r"D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\female"
    dataset_base_path = r"D:\golfDataset\dataset\train"
elif TrainValid == "Validation":
    base_path = r"D:\golfDataset\스포츠 사람 동작 영상(골프)\Validation\Public\female"
    dataset_base_path = r"D:\golfDataset\dataset\test"
else:
    raise ValueError("TrainValid는 'Training' 또는 'Validation'이어야 합니다.")

# 이하 기존 코드
import os
import json
import shutil

# Training/Validation, Public/Assosiation/Amature, male/female 로 경로 나뉨
# base_path, dataset_base_path는 위에서 자동 설정됨
# test, train은 직접 지정

# 기준
true_evaluations = {"best", "good", "normal"}
false_evaluations = {"bad", "worst"}

# dataset 디렉토리 구조 생성
for dataset in ["true", "false"]:
    for ext in ["json", "jpg"]:
        os.makedirs(os.path.join(dataset_base_path, dataset, ext), exist_ok=True)

# JSON 탐색 (라벨 폴더에서만)
for root, dirs, files in os.walk(base_path):
    if "[라벨]" not in root:
        continue

    for file in files:
        if not file.endswith(".json"):
            continue

        json_path = os.path.join(root, file)

        # 대응되는 JPG 경로 생성
        relative_parts = os.path.relpath(json_path, base_path).split(os.sep)
        relative_parts[0] = relative_parts[0].replace("[라벨]", "[원천]")
        jpg_filename = os.path.splitext(file)[0] + ".jpg"
        jpg_path = os.path.join(base_path, *relative_parts[:-1], jpg_filename)

        # JSON 열기 및 evaluation 추출
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                evaluation = data.get("image", {}).get("evaluation", "").lower()
        except Exception as e:
            print(f"Error reading {json_path}: {e}")
            continue

        if evaluation in true_evaluations:
            tf_label = "true"
        elif evaluation in false_evaluations:
            tf_label = "false"
        else:
            print(f"Unknown evaluation '{evaluation}' in {json_path}")
            continue

        # JSON 복사
        shutil.copy2(json_path, os.path.join(dataset_base_path, tf_label, "json", file))

        # JPG 복사 (존재 확인)
        if os.path.exists(jpg_path):
            shutil.copy2(jpg_path, os.path.join(dataset_base_path, tf_label, "jpg", jpg_filename))
        else:
            print(f"Missing JPG for {json_path} → expected {jpg_path}")

print("분류 및 복사 완료.")


분류 및 복사 완료.
